# GAMOS Geometry file creation

GAMOS requires a geometry file to define the structure of world used in each simulations. When optical properties are needed for the simulation, they must be defined for each material at every wavelength of interest. This is accomplished through material property tables. Writing these tables manually is time-consuming and prone to input errors. The following code blocks are used to read an Excel file and create a multi-layer geometry with a variety of properties. The code is currently limited to BOX geometries.

In [ ]:
import os
import pandas
import numpy
# Define input and output directories
input_dir = '../data/'
output_dir = '../results/'

In [ ]:
def print_section_break(title, output_str=''):
    output_str += '\n//{}//\n'.format('-'*76)
    output_str += '// {}\n'.format(title.upper())
    return output_str
def print_materail_init(materials, output_str=''):
    for each_material in sorted(materials):
        
        all_keys = list(materials[each_material]['key'])
        name_index = all_keys.index('name')
        if not materials[each_material]['value'][name_index].startswith('G4_'):
            density_index = all_keys.index('density')
            mixc_index = all_keys.index('mixture_content')
            mixr_index = all_keys.index('mixture_ratio')


            material_names = materials[each_material]['value'][mixc_index].split(',')
            num_materials = len(material_names)
            material_ratios = materials[each_material]['value'][mixr_index].split(',')


            output_str += '\n:MIXT\t{}\t{}\t{}\n'.format(materials[each_material]['value'][name_index],
                                                       materials[each_material]['value'][density_index],
                                                       num_materials)
            i=0
            for each_name in material_names:
                output_str += '{}\t{}\n'.format(each_name,material_ratios[i])
                i += 1
    output_str += '\n'
    return output_str
    
def get_property_list(input_series, delim):
    cur_list = list(input_series.round(7).values.astype(str))
    output_str = delim.join(cur_list)
    return output_str
    
def print_material_properties(materials, material_properties, search='tissue', output_str=''):
    
    for each_material in sorted(materials):
        if search in each_material:
            all_keys = list(materials[each_material]['key'])
            mpt_index = all_keys.index('mpt_name')
            cur_mpt_name = materials[each_material]['value'][mpt_index]

            name_index = all_keys.index('name')
            cur_material = materials[each_material]['value'][name_index]

            all_wavelengths_str = get_property_list(material_properties[each_material]['mpt_wavelength'], '*wvnm ')
            all_rindex_str = get_property_list(material_properties[each_material]['mpt_rindex'], ' ')

            output_str += '\n:MATE_PROPERTIES_TABLE\t{}\n'.format(cur_mpt_name)
            output_str += ':MATEPT_ADD_WAVELENGTHS\t{}\t{}*wvnm\n'.format(cur_mpt_name, all_wavelengths_str)
            output_str += ':MATEPT_ADD_PROPERTY\t{}\tRINDEX\t{}\n'.format(cur_mpt_name,all_rindex_str )
            if 'mpt_miescat' in material_properties[each_material].columns:
                all_mus_str = get_property_list(material_properties[each_material]['mpt_miescat'], '*/mm ')
                output_str += ':MATEPT_ADD_PROPERTY\t{}\tMIE_SCATCOEF\t{}*/mm\n'.format(cur_mpt_name,all_mus_str )
            if 'mpt_abs' in material_properties[each_material].columns:
                all_mua_str = get_property_list(material_properties[each_material]['mpt_abs'], '*/mm ')
                output_str += ':MATEPT_ADD_PROPERTY\t{}\tABSCOEF\t{}*/mm\n'.format(cur_mpt_name,all_mua_str )
            if 'mpt_g' in material_properties[each_material].columns:
                all_gval_str = get_property_list(material_properties[each_material]['mpt_g'], ' ')
                output_str += ':MATEPT_ADD_PROPERTY\t{}\tMIE_GVALUE\t{}\n'.format(cur_mpt_name,all_gval_str )
            if 'mpt_fl_abs' in material_properties[each_material].columns:
                all_fl_absval_str = get_property_list(material_properties[each_material]['mpt_fl_abs'], '*/mm ' )
                output_str += ':MATEPT_ADD_PROPERTY\t{}\tFLUOR_ABSCOEF\t{}*/mm\n'.format(cur_mpt_name,all_fl_absval_str )
            if 'mpt_fl_ems' in material_properties[each_material].columns:
                all_fl_emsval_str = get_property_list(material_properties[each_material]['mpt_fl_ems'], ' ')
                output_str += ':MATEPT_ADD_PROPERTY\t{}\tFLUOR_EMISSION\t{}\n'.format(cur_mpt_name,all_fl_emsval_str )
                output_str += ':MATEPT_ADD_CONST_PROPERTY\t{}\tFLUOR_QUANTUMYIELD\t{:1.1f}\n'.format(cur_mpt_name, float(qys[each_material]))
                output_str += ':MATEPT_ADD_CONST_PROPERTY\t{}\tFLUOR_LIFETIME\t{}\n'.format(cur_mpt_name, lifetimes[each_material]) 
                                                                                                   
                                                                                                   
            output_str += ':MATEPT_ATTACH_TO_MATERIAL\t{}\t{}\n'.format(cur_mpt_name, cur_material)
    output_str += '\n'
    return output_str
    
def print_build_and_place_volumes(volumes, output_str=''):
    rm_name = 'RM00'
    output_str += ':ROTM\t{}\t0.\t0.\t0.\n'.format(rm_name)
    for each_name in volumes['name'].values:
        #print(each_volume)
        cur_volume = all_volumes[all_volumes['name']==each_name]
        geom = cur_volume['geometry'].values[0]
        x = cur_volume['x'].values[0]
        x_unit = cur_volume['x_unit'].values[0]
        y = cur_volume['y'].values[0]
        y_unit = cur_volume['y_unit'].values[0]
        z = cur_volume['z'].values[0]
        z_unit = cur_volume['z_unit'].values[0]
        material = cur_volume['material'].values[0]
        vis = cur_volume['vis'].values[0]
        output_str += '\n:VOLU\t{}\t{}\t{}*{}\t{}*{}\t{}*{}\t{}\n'.format(each_name,
                                                                          geom,
                                                                          x,x_unit,
                                                                          y,y_unit,
                                                                          z,z_unit,
                                                                          material)
        output_str += ':VIS\t{}\t{}\n'.format(each_name, vis.upper())
        
        if each_name != 'world':
            place_x = cur_volume['place_x'].astype(float).values[0]
            place_x_unit = cur_volume['place_x_unit'].values[0]
            place_y = cur_volume['place_y'].astype(float).values[0]
            place_y_unit = cur_volume['place_y_unit'].values[0]
            place_z = cur_volume['place_z'].astype(float).values[0]
            place_z_unit = cur_volume['place_z_unit'].values[0]
            mother = cur_volume['mother'].values[0]
            
            
            output_str += ':PLACE\t{}\t1\t{}\t{}\t{:0.2f}*{}\t{:0.2f}*{}\t{:0.2f}*{}\n'.format(each_name,
                                                                                            mother,
                                                                                            rm_name,
                                                                                            place_x, place_x_unit,
                                                                                            place_y, place_y_unit,
                                                                                            place_z, place_z_unit)
    return output_str
        

In [ ]:
# Define input Excel file
raw_file = os.path.join(input_dir, 'PtG4_MC_geometry_LaRochelle.xlsx')
# First few rows have generic material properties 
#which are not wavelength-dependent
header_row = 7
xls_raw = pandas.ExcelFile(raw_file)
all_materials = {}
all_materials_meta = {}
active_sheets =[]
lifetimes = {}
qys = {}

# Each material has it's own sheet in the Excel which define the properties
# The keywords 'skin', 'fat', or 'muscle' must be in the sheet name
# If the keyword 'tumor' is in the sheet name, additional fluoresence properties will be parsed
# If 'air' is in the sheet name, less properties are needed
# The keyword 'volumes' should only be in one sheet name. This defines the size and placement of the BOX geometries
for each_sheet_name in xls_raw.sheet_names:
    if ('skin' in each_sheet_name) or ('fat' in each_sheet_name) or ('muscle' in each_sheet_name):
        active_sheets.append(each_sheet_name)
        all_materials[each_sheet_name] = xls_raw.parse(each_sheet_name, header=header_row-1)
        all_materials_meta[each_sheet_name] = xls_raw.parse(each_sheet_name, usecols=[0,1], skip_footer=all_materials[each_sheet_name].shape[0]+1)
    elif 'tumor' in each_sheet_name:
        all_materials[each_sheet_name] = xls_raw.parse(each_sheet_name, header=header_row+2-1)
        all_materials_meta[each_sheet_name] = xls_raw.parse(each_sheet_name, usecols=[0,1], skip_footer=all_materials[each_sheet_name].shape[0]+1)
        lifetimes[each_sheet_name] = all_materials_meta['tumor'][all_materials_meta['tumor']['key']=='mpt_lifetime']['value'].values[0]
        qys[each_sheet_name] = all_materials_meta['tumor'][all_materials_meta['tumor']['key']=='mpt_qy']['value'].values[0]
    elif 'air' in each_sheet_name:
        all_materials[each_sheet_name] = xls_raw.parse(each_sheet_name, header=header_row-2-1)
        all_materials_meta[each_sheet_name] = xls_raw.parse(each_sheet_name, usecols=[0,1], skip_footer=all_materials[each_sheet_name].shape[0]+1)
    elif 'volumes' in each_sheet_name:
        all_volumes = xls_raw.parse(each_sheet_name)
        


In [ ]:
all_volumes[all_volumes['name']=='world']

In [ ]:
all_volumes['name'].values

In [ ]:
all_materials_meta.keys()

In [ ]:
all_materials['skin1'].columns

In [ ]:
all_materials['tumor'].columns

In [ ]:
# Create a single string defining the geometry file
file_str = ''
# Create custom materials of custom density by mixing predefined matirals at specific ratios
file_str = print_section_break('Initialize the materials', output_str=file_str)
file_str = print_materail_init(all_materials_meta, output_str=file_str)
# Define the optical properties of all skin layes
file_str = print_section_break('Define skin properties (Layers 1-7)', output_str=file_str)
file_str = print_material_properties(all_materials_meta, all_materials, search='skin', output_str=file_str)
# Define optical properties of fat and muscle layers
file_str = print_section_break('Define fat and muscle', output_str=file_str)
file_str = print_material_properties(all_materials_meta, all_materials, search='fat', output_str=file_str)
file_str = print_material_properties(all_materials_meta, all_materials, search='muscle', output_str=file_str)
# Define tumor optical properties, including fluoresence
file_str = print_section_break('Define tumor inclusion properties\n', output_str=file_str)
file_str = print_material_properties(all_materials_meta, all_materials, search='tumor', output_str=file_str)
# Add air optical properties
file_str = print_section_break('Define general world optical properties\n', output_str=file_str)
file_str = print_material_properties(all_materials_meta, all_materials, search='air', output_str=file_str)
# Define material shape and placements
file_str = print_section_break('Build and place volumes', output_str=file_str)
file_str = print_build_and_place_volumes(all_volumes, output_str=file_str)

In [ ]:
print(file_str)

In [ ]:
# Write the above sting to an output file
# Note: For the simulations in the corresponding manuscript the line
#  :VOLU	tumor_inclusion	BOX	0.5*cm	0.5*cm	5.0*mm	tissue_tumor
# was replaced with
#  :VOLU	tumor_inclusion	ORB	5.0*mm	tissue_tumor
with open(os.path.join(output_dir, 'world.geom'), 'w') as f:
    f.write(file_str)
          